<a href="https://colab.research.google.com/github/seyonechithrananda/bert-loves-chemistry/blob/master/21_HuggingFace_ChemBERTa_SMILES_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Tensorflow 1.x to allow for RDKit to work (ignore this)
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


# Tutorial Part 21: Finetuning HuggingFace's RoBERTa for masked language modelling of SMILES

![alt text](https://huggingface.co/front/assets/huggingface_mask.svg)

By Seyone Chithrananda

Deep learning for chemistry and materials science remains a novel field with lots of potiential. However, the popularity of transfer learning based methods in areas such as NLP and computer vision have not yet been effectively developed in computational chemistry + machine learning. Using HuggingFace's suite of models and the ByteLevel tokenizer, we are able to train a large-transformer model, RoBERTa, on a large corpus of 100k SMILES strings from a commonly known benchmark chemistry dataset, ZINC.

Training RoBERTa over 5 epochs, the model achieves a pretty good loss of 0.398, and may likely continue to decrease if trained for a larger number of epochs. The model can predict tokens within a SMILES sequence/molecule, allowing for variants of a molecule within discoverable chemical space to be predicted.

By applying the representations of functional groups and atoms learned by the model, we can try to tackle problems of toxicity, solubility, drug-likeness, and synthesis accessibility on smaller datasets using the learned representations as features for graph convolution and attention models on the graph structure of molecules, as well as fine-tuning of BERT. Finally, we propose the use of attention visualization as a helpful tool for chemistry practitioners and students to quickly identify important substructures in various chemical properties.

Additionally, visualization of the attention mechanism have been seen through previous research as incredibly valuable towards chemical reaction classification. The applications of open-sourcing large-scale transformer models such as RoBERTa with HuggingFace may allow for the acceleration of these individual research directions.

A link to a repository which includes the training, uploading and evaluation notebook (with sample predictions on compounds such as Remdesivir) can be found [here](https://github.com/seyonechithrananda/bert-loves-chemistry). All of the notebooks can be copied into a new Colab runtime for easy execution.

For the sake of this tutorial, we'll be fine-tuning RoBERTa on a small-scale molecule dataset, to show the potiential and effectiveness of HuggingFace's NLP-based transfer learning applied to computational chemistry.

Install HuggingFace's `transformers` tool, as well as RDKit to visualize molecule designs.

In [1]:
!pip uninstall -y tensorflow
!pip install transformers

In [3]:
# In order for this tutorial to work in Colab, we need to use some nifty magic wget + bash commands to set up a conda environment with RDKit. 
# Installing everything will take roughly 2-3 minutes, so don't worry if it takes some time!

!wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
!chmod +x Anaconda3-2019.10-Linux-x86_64.sh
!bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c rdkit -c conda-forge -c omnia 
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')


--2020-04-14 01:50:55--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530308481 (506M) [application/x-sh]
Saving to: ‘Anaconda3-2019.10-Linux-x86_64.sh’

Anaconda3-2019.10-L 100%[===================>] 505.74M  67.7MB/s    in 7.0s    

2020-04-14 01:51:02 (72.4 MB/s) - ‘Anaconda3-2019.10-Linux-x86_64.sh’ saved [530308481/530308481]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaconda-navigator==1.9.7=py37_0


Since I've uploaded my pretrained RoBERTa model to HuggingFace's model hub, we can utilize HuggingFace's `AutoModelWithLMHead` to load the pre-trained model, and `AutoTokenizer` to load the pre-trained tokenizer. `Pipeline` will help us load the model, tokenizer on masked language prediction. 


In [4]:
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

model = AutoModelWithLMHead.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

Now, to ensure our model demonstrates an understanding of chemical syntax and molecular structure, we'll be testing it on predicting a masked token/character within the SMILES molecule for Remdesivir.

In [5]:
remdesivir_mask = "CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=<mask>1"
remdesivir = "CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=C1"

"CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=O1"


print(fill_mask(remdesivir_mask))

[{'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=C1</s>', 'score': 0.5986586809158325, 'token': 39}, {'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=O1</s>', 'score': 0.09766950458288193, 'token': 51}, {'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=N1</s>', 'score': 0.07694468647241592, 'token': 50}, {'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=21</s>', 'score': 0.0241263248026371, 'token': 22}, {'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=H1</s>', 'score': 0.0188530795276165, 'token': 44}]


Here, we get some interesting results. The final branch, `C1=CC=CC=C1`, is a  benzene ring. Since its a pretty common molecule, the model is easily able to predict the final double carbon bond with a score of 0.60. Let's get a list of the top 5 predictions (including the target, Remdesivir), and visualize them (with a highlighted focus on the beginning of the final benzene-like pattern). Lets import some various RDKit packages to do so.


In [6]:
import torch
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import rdFMCS
from matplotlib import colors
from rdkit.Chem import Draw
from rdkit.Chem.Draw import MolToImage
from PIL import Image


def get_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    Chem.Kekulize(mol)
    return mol


def find_matches_one(mol,submol):
    #find all matching atoms for each submol in submol_list in mol.
    match_dict = {}
    mols = [mol,submol] #pairwise search
    res=rdFMCS.FindMCS(mols) #,ringMatchesRingOnly=True)
    mcsp = Chem.MolFromSmarts(res.smartsString)
    matches = mol.GetSubstructMatches(mcsp)
    return matches

#Draw the molecule
def get_image(mol,atomset):    
    hcolor = colors.to_rgb('green')
    if atomset is not None:
        #highlight the atoms set while drawing the whole molecule.
        img = MolToImage(mol, size=(600, 600),fitImage=True, highlightAtoms=atomset,highlightColor=hcolor)
    else:
        img = MolToImage(mol, size=(400, 400),fitImage=True)
    return img

    

ModuleNotFoundError: ignored

In [0]:
sequence = f"CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC={tokenizer.mask_token}1"
substructure = "CC=CC"
image_list = []

input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input)[0]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
  smi = (sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))
  print (smi)
  smi_mol = get_mol(smi)
  substructure_mol = get_mol(substructure)
  if smi_mol is None: # if the model's token prediction isn't chemically feasible
    continue
  Draw.MolToFile(smi_mol, smi+".png")
  matches = find_matches_one(smi_mol, substructure_mol)
  atomset = list(matches[0])
  img = get_image(smi_mol, atomset)
  img.format="PNG" 
  image_list.append(img)

In [0]:
imfrom IPython.display import Image 

for img in image_list:
  display(img)

As we can see above, 2 of 4 of the model's MLM predictions are chemically valid. The one the model would've chosen (with a score of 0.6), is the first image, in which the top left molecular structure resembles the benzene found in the therapy Remdesivir. Overall, the model seems to understand syntax with a pretty decent degree of certainity. 

However, further training on a more specific dataset (say leads for a specific target) may generate a stronger MLM model. Let's now fine-tune our model on a dataset of our choice. [fine-tuning still needs to be fleshed out]

# Fine-tuning ChemBERTa on a Small Mollecular Dataset

In [0]:
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification
import torch
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

#tokenizer = RobertaTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1', max_length= 512)
model = RobertaForSequenceClassification.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
config = RobertaConfig.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')


In [0]:
dataset_path = "/content/drive/My Drive/Project De Novo/tox21_balanced_revised.csv"

In [4]:
df = pd.read_csv(dataset_path, sep = ',', warn_bad_lines=True, header=None)

df.head()

print(df.columns)

Int64Index([0, 1, 2], dtype='int64')


In [0]:
df.rename(columns={0:'labels',1:'tox_id',2:'smiles'}, inplace=True)

In [7]:
df.head()

,labels,tox_id,smiles
0,0,TOX7137,CCCCCCCC/C=C\CCCCCCCC(N)=O
1,0,TOX5403,CCCCCCOC(=O)c1ccccc1
2,0,TOX17626,O=C(c1ccc(Cl)cc1)c1ccc(Cl)cc1
3,0,TOX20718,COc1cc(Cl)c(OC)cc1N
4,0,TOX26989,N[C@H](Cc1c[nH]c2ccccc12)C(=O)O


In [8]:
smiles = []
labels = []

smiles = df['smiles'].to_list()
labels = df['labels'].to_list()

#check if both data and labels are alligned

print (df.tail())

# Check if data is alligned
print(smiles[2139])
print(labels[2139])

print(type(smiles))
print(type(labels))

print(len(smiles))
print(len(labels))

      labels    tox_id                                             smiles
2137       0   TOX6864                                     CCCCCCCC(=O)OC
2138       0   TOX3377                       CN1Cc2c(N)cccc2C(c2ccccc2)C1
2139       1  TOX28911  CCC(=O)O[C@H]1CC[C@H]2[C@@H]3CCc4cc(O)ccc4[C@H...
2140       0   TOX2842                                       Cc1ncsc1CCCl
2141       0  TOX27035                                    CCCCCCC/C=C/C=O
CCC(=O)O[C@H]1CC[C@H]2[C@@H]3CCc4cc(O)ccc4[C@H]3CC[C@]12C
1
<class 'list'>
<class 'list'>
2142
2142


In [0]:
#Write list of SMILES to .txt file for tokenizer to read

with open("smiles_tox21.txt", "w") as fobj:
    for x in smiles:
        fobj.write(x + "\n")


In [0]:
#Train new tokenizer --> pre-trained tokenizer doesn't seem to transfer to tox21 well and misses new vocab
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train("/content/smiles_tox21.txt", vocab_size=52000, min_frequency=2, special_tokens=["<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"])

In [11]:
labels = torch.tensor(labels).unsqueeze(dim=0)
print (labels.shape)

torch.Size([1, 2142])


In [12]:
print(type(smiles))
print (type(labels))
print(smiles)
tokenized_smiles = tokenizer.encode(smiles)
input_smiles = torch.tensor(tokenized_smiles).unsqueeze(0)

<class 'list'>
<class 'torch.Tensor'>
['CCCCCCCC/C=C\\CCCCCCCC(N)=O', 'CCCCCCOC(=O)c1ccccc1', 'O=C(c1ccc(Cl)cc1)c1ccc(Cl)cc1', 'COc1cc(Cl)c(OC)cc1N', 'N[C@H](Cc1c[nH]c2ccccc12)C(=O)O', 'Cc1ccccc1N1C(=O)c2cc(S(N)(=O)=O)c(Cl)cc2NC1C', 'CCCC(CCC)C(=O)[O-]', 'CC(=O)O[Si](C)(OC(C)=O)OC(C)=O', 'CCC(C)C1N=C(C)C(C)S1', 'C=CCCCCCCCC', 'O=P([O-])([O-])OC(CO)CO', 'O=C(O)c1cccc(C(=O)O)n1', 'O=[N+]([O-])C=Cc1ccccc1', 'N#Cc1c[nH]cc1-c1cccc2c1OC(F)(F)O2', 'O=C(O)Cn1c(C(=O)Nc2nc(-c3ccccc3Cl)cs2)cc2ccccc21', 'CCCC1CCC(=O)O1', 'c1ccc2cnncc2c1', 'O=[N+]([O-])c1cc(C(F)(F)F)cc([N+](=O)[O-])c1Cl', 'C/C=C1\\NC(=O)[C@H]2CSSCCC=C[C@H](CC(=O)N[C@H](C(C)C)C(=O)N2)OC(=O)[C@H](C(C)C)NC1=O', 'CC(Cl)CCl', 'CC(=O)OCCC(C)CC(C)(C)C', 'COc1cc(CCC(C)=O)ccc1O', 'O=C(N/N=C/c1ccc([N+](=O)[O-])o1)c1cc([N+](=O)[O-])cc([N+](=O)[O-])c1O', 'N#Cc1c(Cl)c(Cl)c(Cl)c(C#N)c1Cl', 'O=C(CBr)c1ccccc1', 'C[C@]12O[C@H](C[C@]1(O)CO)n1c3ccccc3c3c4c(c5c6ccccc6n2c5c31)CNC4=O', 'CC(=O)Nc1c(I)c(C(=O)N[C@H]2C(O)O[C@H](CO)[C@@H](O)[C@@H]2O)c(I)c(N(

TypeError: ignored

In [0]:
print(input_smiles.shape)
print(labels.shape)
print(len(tokenized_smiles))

In [11]:
outputs = model(input_smiles, labels=labels)
loss, logits = outputs[:2]

print('Loss: '+ loss)
print('Logits: '+ logits)


RuntimeError: ignored

In [0]:

train_size=0.8
train_dataset=df.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


In [0]:
max_epochs = 3
model = model.train()
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        optimizer.zero_grad()
        sent = sent.squeeze(0)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
        output = model.forward(sent)[0]
        _, predicted = torch.max(output, 1)
        
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        if i%100 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(0)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))